In [43]:
import pandas as pd
# import numpy as np
import kaggle
# import customfunctions as cf
import re


## Download dataset TV Time from Kaggle 

using Kaggle API

In [2]:
kaggle.api.authenticate()
kaggle.api.dataset_download_files('oscarfry/tvtime-shows', path='./data/tvtimeshows')

In [38]:
df_tvtime = pd.read_csv('data/tvtimeshows/tvtimeshows.csv')

## Generate descriptive statistics

In [4]:
df_tvtime.describe(include='all')

,id,name,followers,nb_rates,runtime,number_of_seasons,network,mean_rate,poster_image,seasons
count,1.981600e+04,19812,1.981600e+04,19816.000000,19816.000000,19816.000000,18854,19816.000000,19816,19816
unique,NaN,19479,NaN,NaN,NaN,NaN,945,NaN,18658,5300
top,NaN,***Movies are not allowed***,NaN,NaN,NaN,NaN,YouTube,NaN,https://d36rlb2fgh8cjd.cloudfront.net/default-...,"[{'number': 1, 'nb_episodes': 12}]"
freq,NaN,6,NaN,NaN,NaN,NaN,715,NaN,1159,1061
mean,2.670577e+05,NaN,1.036959e+04,376.513474,42.138423,9.467198,NaN,3.852017,NaN,NaN
std,3.733336e+05,NaN,9.396993e+04,4044.547313,27.277166,116.659625,NaN,1.008189,NaN,NaN
min,7.032700e+04,NaN,1.630000e+02,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,1.821335e+05,NaN,3.210000e+02,6.000000,25.000000,1.000000,NaN,3.630000,NaN,NaN
50%,2.875910e+05,NaN,7.800000e+02,20.000000,45.000000,1.000000,NaN,4.060000,NaN,NaN
75%,3.320820e+05,NaN,2.993250e+03,86.000000,55.000000,3.000000,NaN,4.470000,NaN,NaN


## Problemas encontrados:

### 1. Existem subsidiária de uma mesma Network
#### Exemplo: 
+ Amazon, Amazon (Japan), Amazon Prime Instant Video
+ Rede Globo, Globoplay, Globosat 

Uniques are returned in order of appearance. 
> `df_tvtime['network'].unique()`

### 2. Não registra mudança de emissoras
#### Exemplo 
COMMUNITY era da NBC da Temporadas 1 a 5, mas na Temporada 6 foi da Yahoo! Screen, mas no dataset consta apenas Yahoo! Screen


In [24]:
df_tvtime[df_tvtime['name'].str.contains('COMMUNITY', flags=re.IGNORECASE, regex=True, na=False)]

,id,name,followers,nb_rates,runtime,number_of_seasons,network,mean_rate,poster_image,seasons
3945,94571,Community,221166,10256,25,6,Yahoo! Screen,3.93,https://dg31sz3gwrwan.cloudfront.net/poster/94...,"[{'number': 1, 'nb_episodes': 25}, {'number': ..."


## Networks relevantes

### * Big Three television networks 

ABC , CBC , NBC

### * [Streamings serve provider](https://en.wikipedia.org/wiki/Streaming_service_provider)

NETFLIX, AMAZON, DISNEY, HULU, YOUTUBE, HBO, CRUNCHYROLL, GLOBO, APPLE


### * TOP 30 ( sem agrupar com subidiarias)

> `df_tvtime['network'].value_counts().head(30)`

FUJI, FOX, AT-X, CARTOON NETWORK, ITV, MTV, NICKELODEAN, DISCOVERY, HISTORY

In [39]:
popular_networks = [ "ABC", "CBC", "NBC", "NETFLIX", "AMAZON", "DISNEY", "HULU", "YOUTUBE", "HBO", "CRUNCHYROLL", "GLOBO", "APPLE", "FOX", "CARTOON NETWORK", "MTV", "NICKELODEAN", "DISCOVERY", "HISTORY" ]



In [45]:
def networks_more_title(df_tt, networks): 
    column_names = list(df_tt.columns)
    df_most_popular = pd.DataFrame(columns = column_names)
    for i in networks:
        df = df_tt[df_tt['network'].str.contains(i, flags=re.IGNORECASE, regex=True, na=False)]
        df_most_popular = df_most_popular.append(df,ignore_index=True)
    return df_most_popular


In [47]:
df_network_popular = networks_more_title(df_tvtime, popular_networks)

In [51]:
df_network_popular['network'].value_counts()

YouTube                       715
Netflix                       656
ABC (US)                      495
NBC                           429
FOX                           288
                             ... 
Fox Action Movies               1
Amazon Prime Instant Video      1
Discovery Kids                  1
CBC (EG)                        1
Fox Crime                       1
Name: network, Length: 110, dtype: int64

In [53]:
df_network_popular.sort_values(by='followers', ascending=False)

,id,name,followers,nb_rates,runtime,number_of_seasons,network,mean_rate,poster_image,seasons
3164,121361,Game of Thrones,4618578,233167,55,8,HBO,4.78,https://dg31sz3gwrwan.cloudfront.net/poster/12...,"[{'number': 1, 'nb_episodes': 10}, {'number': ..."
1371,305288,Stranger Things,4016670,161986,50,4,Netflix,4.81,https://dg31sz3gwrwan.cloudfront.net/poster/30...,"[{'number': 1, 'nb_episodes': 8}, {'number': 2..."
1238,323168,13 Reasons Why,3631287,156627,60,3,Netflix,4.64,https://dg31sz3gwrwan.cloudfront.net/poster/32...,"[{'number': 1, 'nb_episodes': 13}, {'number': ..."
1341,264586,Orange Is the New Black,2919447,101642,60,7,Netflix,4.72,https://dg31sz3gwrwan.cloudfront.net/poster/26...,"[{'number': 1, 'nb_episodes': 13}, {'number': ..."
631,73762,Grey's Anatomy,2689998,102573,45,16,ABC (US),4.55,https://dg31sz3gwrwan.cloudfront.net/poster/73...,"[{'number': 1, 'nb_episodes': 9}, {'number': 2..."
...,...,...,...,...,...,...,...,...,...,...
634,360769,"Scooby Doo, Where Are You! (1978)",163,0,20,3,ABC (US),0.00,https://d36rlb2fgh8cjd.cloudfront.net/default-...,"[{'number': 1, 'nb_episodes': 2}, {'number': 3..."
4152,293316,Looped,163,0,25,1,Cartoon Network,3.98,https://dg31sz3gwrwan.cloudfront.net/poster/29...,"[{'number': 1, 'nb_episodes': 52}]"
4543,357146,Bad Chad Customs,163,0,45,1,Discovery,3.95,https://dg31sz3gwrwan.cloudfront.net/poster/35...,"[{'number': 1, 'nb_episodes': 6}]"
633,72489,"Dynomutt, Dog Wonder",163,8,25,2,ABC (US),4.02,https://dg31sz3gwrwan.cloudfront.net/poster/72...,"[{'number': 1, 'nb_episodes': 16}, {'number': ..."


In [69]:
# df_network_popular['network'].str.contains('GLOBO', re.IGNORECASE).groupby(df_network_popular['runtime']).mean()